In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [5]:
class GAN_image():
    def build_discriminator(self):
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(1,activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=self.img_shape)
        
        validity = model(img)
        
        return Model(img, validity)
    
    def build_generator(self):
        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.latent_dim = 100
        
        optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        #itt taníthatók a discriminator súlyai(compileban úgy szerepel)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        self.generator = self.build_generator()
        
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        self.discriminator.trainable = False
        
        validity = self.discriminator(img)
        
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        

In [6]:
# tanítás paraméterek
steps = 30000
batch_size = 128
sample_interval = 50

In [7]:
gan = GAN_image()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

In [8]:
# adatok betöltése
from keras.datasets import mnist
(x_train, _), (_, _) = mnist.load_data()

# skálázás -1 és 1 közé
x_train = x_train / 127.5 - 1
x_train = np.expand_dims(x_train, axis = 3) # legyen channel dim is

# diszkriminátor kimenetek
valid = np.ones((batch_size, 1)) # D(x)    = 1
fake = np.zeros((batch_size, 1)) # D(G(z)) = 0

In [9]:

from IPython import display

def plot_images(gan, step):
    r, c = 2,5
    noise = np.random.normal(0, 1, (r * c, gan.latent_dim))
    gen_imgs = gan.generator.predict(noise)
  
  # rescale to 0-1 (from -1 -- 1)
    gen_imgs = 0.5 * gen_imgs + 0.5
  
    plt.title('step ' + str(step))
    fig, ax = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            ax[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            ax[i,j].axis('off')
            cnt += 1
  
    display.display(plt.gcf())

In [10]:
# G és D tanítása, kirajzolással

for step in range(5000):
    
  # képekből véletlen batch-nyi kiválasztása
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]
  
  
  # egy batch-nyi kép generálás
    noise = np.random.normal(0, 1, (batch_size, gan.latent_dim))
    gen_imgs = gan.generator.predict(noise)
  
  # diszkriminátor tanítása
    d_loss_real = gan.discriminator.train_on_batch(imgs, valid)
    d_loss_fake = gan.discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
  
  
  # generátor tanítása
    g_loss = gan.combined.train_on_batch(noise, valid)
  
  # log
    if step % 100 == 0:
        print(step, 'loss_D', d_loss[0], 'loss_G', g_loss)
    
  # generált képek kirajzolása
    if step % 100 == 0:
        plot_images(gan, step)
    

InternalError:  Blas GEMM launch failed : a.shape=(32, 100), b.shape=(100, 256), m=32, n=256, k=100
	 [[node sequential_4/dense_11/MatMul (defined at C:\Users\Dani\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_2090]

Function call stack:
keras_scratch_graph
